## Ayudantía 11: Interfaces Gráficas II 💻🧵🌐

### Ayudantes 👾
- Julio Huerta
- Felipe Vidal
- Diego Toledo
- Alejandro Held
- Clemente Campos

### 📖 Contenidos 📖
En esta ayudantía veremos:
- QThreads para implementar *multithreading* en aplicaciones gráficas.
- Integración de PyQt con Networking

### Introducción

Ya aprendimos la semana (ante)pasada a crear aplicaciones con interfaces gráficas. Hablamos de los elementos gráficos fundamentales y de cómo eventos y señales pueden hacer que nuestros elementos se comuniquen. Pero estas aplicaciones eran, dentro de todo, sencillas y **lineales**. Si pensamos en las aplicaciones gráficas que nos rodean, son más complejas, con múltiples cosas sucediendo al mismo tiempo e incluso sin intervención del usuario.

Por ejemplo, cuando jugamos Tetris, no es el usuario quien indica que los *tetrominós* (si, así se llaman) bajen, si no que existe un manejo automático de la caída, que aumenta su velocidad a medida que el juego avanza 🤓.

### Multithreading en GUI's 🧵

Hasta el momento, nuestros programas gráficos son lineales, lo que significa que si queremos modelar la caída del tetrominó (usando, por ejemplo, `time.sleep`), el programa se congelará, pues solo puede procesar esa instrucción y esperará a que termine.

¿La solución? Implementar *threading* con **QThreads**.

### QThread

Un `QThread` de PyQt es el análogo al `Thread` del módulo `threading`. Funciona prácticamente de la misma forma, con la ventaja de que al ser un `QObject` interactúa mejor con el resto de objetos de PyQt:


In [ ]:
from PyQt5.QtCore import QThread, pyqtSignal
from time import sleep


class ThreadTetromino(QThread):

    def __init__(self, senal_mover: pyqtSignal, tiempo):
        super().__init__()
        self.senal_mover = senal_mover
        self.tiempo = tiempo
        
    def run(self):
        while True:
            self.senal_mover.emit()
            sleep(self.tiempo)


Por ejemplo, el código anterior puede corresponder a la parte lógica del movimiento de un tetrominó. La señal, que se le pasa a este objeto, puede estar conectada a una ventana que se encarga de desplazar un `QLabel` asociado a dicho tetrominó.


### Múltiples QThreads

Si queremos modelar un juego en el que tenemos varios tetrominós cayendo al mismo tiempo, probablemente optemos por asignar un `QThread` individual a cada uno de ellos, tal que sean completamente independientes. Pero al hacer esto, debemos tener cuidado de **no perder la referencia al thread**. Esto es, debemos guardarlo de alguna forma, como en una lista:


In [ ]:
from PyQt5.QtCore import QThread, QObject, pyqtSignal
from time import sleep


class ThreadTetromino(QThread):
    senal_mover = pyqtSignal(int, int, int)
    
    def __init__(self, id_tetromino, tiempo, x, y):
        super().__init__()
        self.x = x
        self.y = y
        self.id_tetromino = id_tetromino
        self.tiempo = tiempo
        
    def run(self):
        while True:
            # En este caso, la señal envía además el id del tetrominó que representa este thread
            self.y += 1
            self.senal_mover.emit(id_tetromino, x, y)
            sleep(self.tiempo)


class LogicaTetris(QObject):
    senal_enviar_movimiento = pyqtSignal(int, int, int)
    
    def __init__(self):
        super().__init__()
        self.threads = []
        
    # Digamos que este método es ejecutado al crear un nuevo tetrominó
    def crear_thread(self, id_tetromino):
        thread = ThreadTetromino(2, 0, 0)
        thread.senal_mover.connect(self.enviar_movimiento)
        self.threads.append(thread) # ESTA PARTE ES IMPORTANTE!
        thread.start()
        
    def enviar_movimiento(self, id_tetromino, x, y):
        # Esta señal puede estar conectada con un método del front-end
        senal_enviar_movimiento.emit(id_tetromino, x, y) 
        
        

### QMutex 🔒

Si `QThread` es el símil del `Thread` en PyQt, `QMutex` es el símil del `Lock`. Se utiliza de la misma manera, y con el mismo propósito: Evitar que dos `QThreads` accedan a la misma sección crítica de código al mismo tiempo.

Para no preocuparnos de las posiciones, en vez de usar tetrominós, usaremos monominós (cubitos, representados por un único par (x, y))

In [ ]:
from PyQt5.QtCore import QThread, QObject, QMutex


class ThreadMonomino(QThread):
    
    def __init__(self, id_monoomino, tiempo, x, y):
        super().__init__()
        self.x = x
        self.y = y
        self.id_monomino = id_monomino
        self.tiempo = tiempo
        self.avanzar = True
        
    def run(self):
        while self.avanzar:
            self.y += 1
            self.senal_mover.emit(id_monomino, x, y)
            sleep(self.tiempo)


class LogicaMonomis(QObject):
    
    def __init__(self):
        super().__init__()
        self.tablero = [[None for c in range(10)] for c in range(20)]
        self.threads = {}
        self.mutex = QMutex()
        
    def crear_thread(self, id_monomino):
        thread = ThreadMonomino(2)
        thread.senal_mover.connect(self.enviar_movimiento)
        self.threads[id_monomino] = thread
        thread.start()
        
    def enviar_movimiento(self, id_monomino, x, y):
        # Ahora, el tablero debe actualizarse, pero debe revisarse en caso de colisiones
        # Para evitar errores por concurrencia sobre este tablero, usaremos un QMutex
        self.mutex.lock()
        if self.tablero[x][y] is not None:
            self.threads[id_monomino].avanzar = False
        else:
            self.tablero[x][y] = id_monomino
            senal_enviar_movimiento.emit(id_monomino) 
        self.mutex.unlock()
        
        

### Networking con GUI's

Ya sabemos como hacer aplicaciones *ehem...* "bonitas" con PyQt, y sabemos como hacer que dos programas se comuniquen mediante el uso de sockets y nuestros conocimientos de codificación. Entonces, ¿por qué no juntar estos dos mundos en un solo programa?

Recordemos la arquitectura "Cliente-Servidor". Muchos de sus juegos favoritos se modelan siguiendo esta estructura, pero para que sean llamativos, el lado del cliente usualmente ofrece una interfaz gráfica. Por otro lado, el lado del servidor usualmente no tiene que ser tan bonito, y puede existir mediante una simple consola. 

La integración de una interfaz gráfica en dicho caso, se puede ver en este diagrama:

![Integración de networking con GUI's. El cliente, en su backend, agrega la lógica de conexión hacia el servidor. Por otro lado, el servidor existe como siempre lo ha hecho.](networking-interfaces.png "Integración de networking con GUI's")

## Tetris 99

En este ejemplo aplicado, nos fijaremos puntualmente en los siguientes aspectos:

* `backend/cliente.py`: sera nuestro archivo encargado de manejar la conexión del cliente y la logica de inicio del juego, para esto se implementan los siguientes pasos

    1.  Se inicializa el cliente y se realiza la conexión al servidor (este ultimo ya debe estar corriendo)

    2.  Se implementa un protocolo de comandos para comunicar al cliente con el servidor, de esta forma el Servidor puede responder a diferentes solicitudes del cliente, podemos ver más detalles en los métodos `verificar_usuario` y `enviar_puntaje`

    3.  Al principio del juego el frontend pide un nombre de usuario para iniciar. Este `username` es pasado al cliente por medio de señales y el cliente lo envía al servidor por networking para que el servidor haga el check de sí es un nombre valido (largo no puede ser 0 y no debe tener ",")
    
    4.  Cuando el cliente recibe la respuesta del servidor emite distintas señales segun corresponda:
    
        + Si el username es valido, emite la señal para iniciar el juego, esta desencadena una seríe de acciones:
            - Inicia la logica del juego (backend)
            - esconde la ventana de inicio
            - hace `show()` a la ventana de juego
            - Inicia la musica en el cliente :D
            
        + Si el username es invalido, se emite la señal para mostrar la ventana de error

    
    5. Finalmente cuando el usuario pierde, la ventana de juego le comunica al cliente (mediante señales) el puntaje del usario, el cliente le manda este mensaje al servidor, para que que imprima en un archivo de salida los puntajes obtenidos


* `backend/logica_juego.py`: Este es el encargado de manejar toda la logica del avance de los blockes, ver si se ha hecho una linea para dar el puntaje y posteriormente eliminarla y estar chequeando si el usuario ha perdido. Es un modulo bien denso, pero todo se logra gracias a un unico `QTimer` presente en el método `comenzar_partida`. Esto nos da un ejemplo de lo poderosos que son los `QThreads` y los `Qtimers`


* `servidor.py`: Servidor del networking que escucha conexiones de clientes para validar el nombre de usuario y escribir el archivo de puntajes. Tiene la particularidad que funciona con un `QThread` que permite interacturar de mejor forma con elementos de la biblioteca `PyQt`. Sin embargo no es posible darle `argumentos` a la función target de `QThread` por lo cual se creo uno personalizado.

* `frontend/`: Carpeta que tiene las ventanas de incio, juego y el mensaje de error. Fueron realizadas con un antiguo contenido utilizando archivos `.ui` ya que se le dio más importancia al networking y la interacción con el backend (mediante señales). Es importante mencionar que el uso de archivos `.ui` no esta permitido en las tareas, así que usted no lo haga :


### Sonidos 🔊

En caso de que no se hayan dado cuenta, el cliente del juego Tetris incluye una sección de código que nos permite reproducir sonidos que tengamos almacenados localmente:

In [ ]:
from PyQt5.QtMultimedia import QMediaPlayer, QMediaContent
from PyQt5.QtCore import QUrl
import os

class Musica(QObject):
    
    def __init__(self, ruta_cancion):
        super().__init__()
        self.ruta_cancion = ruta_cancion

    def comenzar(self):
        try:
            self.player = QMediaPlayer(self)
            path = os.path.abspath(self.ruta_cancion)
            if not os.path.isfile(path):
                raise FileNotFoundError(f"El archivo '{path}' no fue encontrado.")
                
            cancion = QMediaContent(QUrl.fromLocalFile(path))
            self.player.setMedia(cancion)
            self.player.play()
            self.player.mediaStatusChanged.connect(self.loopear_cancion)
        
        except FileNotFoundError as error:
            print('Error: archivo de música no encontrado:', error)
        
        except ValueError as error:
            print('Error de valor al cargar el archivo de música:', error)
        
        except RuntimeError as error:
            print('Error en el sistema multimedia:', error)

    def loopear_cancion(self, status):
        if status == QMediaPlayer.EndOfMedia:
            self.player.setPosition(0)
            self.player.play()

Aquí se utiliza `QMediaPlayer` junto a `QMediaContent`. Esto se utiliza cuando tenemos archivos de tipo MP3. Para archivos WAV podemos utilizar `QSoundEffect`(¡más detalle en los contenidos!).

Algo importante que notar es que necesitamos crear un `QUrl` para cargar correctamente la canción, que utiliza el **path absoluto** del recurso. Por fortuna, esto es fácil de lograr usando la función `os.path.abspath` junto al path relativo de la canción. 

Además, el objeto `QMediaPlayer` tiene un evento `mediaStatusChanged` que se lanza al cambiar el estado de la reproducción (por ejemplo, cuando la canción se acaba). Podemos aprovecharlo para reaccionar a cuando la canción acaba y reiniciar el reproductor.